# Assignment 1 -- Unit Commitment
## Optimisation
### Morgan Reilly, Shaurabh Kumar Singh 
### 20235398, (student number Shurabh)

### References
* https://www.coursera.org/lecture/python-analysis/tabular-data-as-a-nested-list-57msf

In [17]:
from ortools.linear_solver import pywraplp
import matplotlib.pyplot as plt
import pandas as pd
import csv

In [18]:
# Instantiate Solver
solver = pywraplp.Solver('Generator', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

<!-- Variables -->
$\underline{\textbf{Variables}}$
<br><br>
$x$ -> $x^{H} = generators$ 
<br>
$y$ -> $y^{H} = hour$
<br><br>
$A_{y} = $ Energy produced by Hydro Generator $A$ in $y^{h}$ hours<br>
$B_{y} = $ Energy produced by Hydro Generator $B$ in $y^{h}$ hours<br>
$C_{y} = $ Energy produced by Hydro Generator $C$ in $y^{h}$ hours<br>
$D_{y} = $ Energy produced by Hydro Generator $D$ in $y^{h}$ hours<br>
$E_{y} = $ Energy produced by Solid Generator $E$ in $y^{h}$ hours<br>
$F_{y} = $ Energy produced by Solid Generator $F$ in $y^{h}$ hours<br>
$G_{y} = $ Energy produced by Solid Generator $G$ in $y^{h}$ hours<br>
$H_{y} = $ Energy produced by Gas Generator $H$ in $y^{h}$ hours<br>
$I_{y} = $ Energy produced by Solar Generator $I$ in $y^{h}$ hours<br>
$J_{y} = $ Energy produced by Solar Generator $J$ in $y^{h}$ hours<br>

<!-- Constraints -->
$\underline{\textbf{Constraints}}$
<br><br>
$ \sum_{i=1}^{10} P_{i}\times C_{i} = Demand[j] $ 
<br><br>
&nbsp;&nbsp;&nbsp;&nbsp; $ where:$
* ${i} = A, B, C,..., J$ -> $Generators$
* ${j} = 1, 2, 3,..., 24$ -> $Hours$
* $P = P_{1},..., P_{10}$ -> $Power$
* $C = C_{1},..., C_{10}$ -> $Cost$
* $T = T_{1},..., T_{24}$ -> $Time$

<br>
$Non-Solar$ $Generators:$<br><br>&nbsp;&nbsp;&nbsp;&nbsp;
$Lower$ $Bound[i] \leqslant E_{i}{j} \leqslant Upper$ $Bound[i]$
<br><br>
$Solar$ $Generators:$<br><br>&nbsp;&nbsp;&nbsp;&nbsp;
$Lower$ $Bound[i] \leqslant E_{i}{j} \leqslant Upper$ $Bound[i]\times Solar$ $Curve[j]$

<br><br>
<br>
$ \int_{100}^{10} \! A\times{1.4} + 
    \int_{80}^{10} \! B\times{1.4} +
    \int_{60}^{10} \! C\times{1.4} +
    \int_{10}^{1} \! D\times{1.4} +
    \int_{900}^{100} \! E\times{4.4} +
    \int_{600}^{100} \! F\times{4.4} +
    \int_{100}^{10} \! G\times{4.4} +
    \int_{400}^{100} \! H\times{9.1} +
    \int_{70}^{0} \! I\times{6.6} +
    \int_{20}^{0} \! J\times{6.6} \leqslant Minimum
$ <br>

<!-- Objective Function -->
$\underline{\textbf{Objective Function}}$
<br><br>
$Minimise$&nbsp;$Cost:$ <br><br>&nbsp;&nbsp;&nbsp;&nbsp;
$ \sum_{i=A}^{J} (E_{i}{j}\times C_{i})\times C_{i}$&nbsp;&nbsp;&nbsp;$\forall$ &nbsp; ${[j = 1,...,24]} $

<br><br>
* From $1^{st}$ to $6^{th}$ hour:
    * $Hydro$:
        * $H_{1}$, $H_{2}$, $H_{3}$, $H_{4}$
    * $Solid$:
        * $Solid_{1}$, $Solid_{2}$, $Solid_{3}$
    * $Gas$:
        * $G_{1}$
    * $Solar$:
        * $Solar_{1}$, $Solar_{2}$
<br><br>

### Decision Variables
* In a small country there are 10 generators of four types: Hydroelectric, Solid fuel, Gas, Solar.
* Each generator has a lower and upper bound on its production per hour (in MW/h). Between these bounds, production is not limited to discrete levels. 
* Each generator also has a cost for producing each MW, and produces a certain amount of CO2 per MW.

In [19]:
#Ingest the Input
generators = pd.read_csv('generator_info.csv')
generators.head()

,name,type,lower_bound (MW),upper_bound (MW),cost/MW,CO2/MW
0,A,hydro,10,100,1.4,0.024
1,B,hydro,10,80,1.4,0.024
2,C,hydro,10,60,1.4,0.024
3,D,hydro,1,10,1.4,0.024
4,E,solid,100,900,4.4,0.820


In [20]:
generators.columns = ["name","type","lower_bound (MW)","upper_bound (MW)", "cost/MW", "CO2/MW"]
lower_bound = generators["lower_bound (MW)"].values.tolist()
upper_bound = generators["upper_bound (MW)"].values.tolist()
cost = generators["cost/MW"].values.tolist()
print("Lower Bounds: ", lower_bound)
print("Upper Bounds: ", upper_bound)
print("Costs: ", cost)

Lower Bounds:  [10, 10, 10, 1, 100, 100, 10, 100, 0, 0]
Upper Bounds:  [100, 80, 60, 10, 900, 600, 100, 400, 70, 20]
Costs:  [1.4, 1.4, 1.4, 1.4, 4.4, 4.4, 4.4, 9.1, 6.6, 6.6]


In [21]:
#Read the Demand.csv file
demand_df = pd.read_csv('demand.csv', header=None, names=['mw'])
demand_df.head()

,mw
0,1461
1,1446
2,1446
3,1438
4,1425


In [22]:
#Read the Solar Curve.csv file
solar_df = pd.read_csv('solar_curve.csv', header=None, names=['sw'])
solar_df.head()

,sw
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [23]:
#Create Continous variables for each generator and each hour in a day with their lower and upper bounds
A = [solver.NumVar(lower_bound[0], upper_bound[0], f'A{i}') for i in range(0,24)]
B = [solver.NumVar(lower_bound[1], upper_bound[1], f'B{i}') for i in range(0,24)]
C = [solver.NumVar(lower_bound[2], upper_bound[2], f'C{i}') for i in range(0,24)]
D = [solver.NumVar(lower_bound[3], upper_bound[3], f'D{i}') for i in range(0,24)]
E = solver.NumVar(lower_bound[4], upper_bound[4], "E")
F = solver.NumVar(lower_bound[5], upper_bound[5], "F")
G = solver.NumVar(lower_bound[6], upper_bound[6], "G")
H = [solver.NumVar(lower_bound[7], upper_bound[7], f'H{i}') for i in range(0,24)]
I = [solver.NumVar(lower_bound[8], upper_bound[8]*solar_df['sw'][i], f'I{i}') for i in range(0,24)]
J = [solver.NumVar(lower_bound[9], upper_bound[9]*solar_df['sw'][i], f'J{i}') for i in range(0,24)]

### Constraints
* The maximum supply from any Solar generator depends on the time of day. Relative to the generator’s maximum, it can achieve 50% from 6am to 10am, 100% from 11am to 3pm, 50% from 4pm to 6pm, and 0% otherwise.
* The Solid fuel generators cannot change their amount of production from one hour to the next.

TODO: Display contraints here in LaTeX form

In [24]:
#Creating Contraints
#1. Energy generated per hour is equal to the per hour demand
for i in range(0, 24):
    solver.Add(A[i] + B[i] + C[i] + D[i] + E + F + G + H[i] + I[i] + J[i] == demand_df['mw'][i]) #for each hour
    
#print(solver.NumConstraints())

### Formaising Objective
* Goal is to meet demand at each hour at the minimum cost, by choosing how much energy each generator should produce per hour
* The demand from electricity customers varies by hour of the day. This is given in demand.csv. If we over-supply relative to demand, it can damage the electricity infrastructure. If we under-supply, then some customers can’t boil their kettles.

TODO: Display formalised objective here in LaTeX form

In [25]:
#Objective
objective = solver.Objective()

for i in range(0, 24):
    objective.SetCoefficient(A[i],cost[0])
    objective.SetCoefficient(B[i],cost[1])
    objective.SetCoefficient(C[i],cost[2])
    objective.SetCoefficient(D[i],cost[3])
    objective.SetCoefficient(H[i],cost[7])
    objective.SetCoefficient(I[i],cost[8])
    objective.SetCoefficient(J[i],cost[9])

objective.SetCoefficient(E, cost[4] * 24)
objective.SetCoefficient(F, cost[5] * 24)
objective.SetCoefficient(G, cost[6] * 24)

objective.SetMinimization()

### Solve

In [26]:
solver.Solve()
print('Objective value =', objective.Value())

for v in solver.variables():
    print(v.name, v.solution_value())

Objective value = 151518.00000000003
<bound method Variable.name of A0> 68.99999999999983
<bound method Variable.name of A1> 53.999999999999886
<bound method Variable.name of A2> 53.999999999999886
<bound method Variable.name of A3> 45.99999999999997
<bound method Variable.name of A4> 32.999999999999886
<bound method Variable.name of A5> 27.999999999999915
<bound method Variable.name of A6> 74.99999999999986
<bound method Variable.name of A7> 100.0
<bound method Variable.name of A8> 100.0
<bound method Variable.name of A9> 100.0
<bound method Variable.name of A10> 100.0
<bound method Variable.name of A11> 100.0
<bound method Variable.name of A12> 100.0
<bound method Variable.name of A13> 100.0
<bound method Variable.name of A14> 100.0
<bound method Variable.name of A15> 100.0
<bound method Variable.name of A16> 100.0
<bound method Variable.name of A17> 100.0
<bound method Variable.name of A18> 100.0
<bound method Variable.name of A19> 100.0
<bound method Variable.name of A20> 100.0
<bo